# Import Libraries

In [ ]:
# For drive access
from google.colab import drive
drive.mount('/content/drive')

# Standard libraries
import numpy as np
import pandas as pd
import time

# For audio
import librosa

# For preprocessing
import tensorflow as tf

# for saving the dataframe to disk
import pickle

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Self Defined Class Methods For Feature Extraction

In [ ]:
class Framed:

  def __init__(self,
               dataframe,
               sr=16000,
               window_size_s=5.0,
               hop_size_s=2.5,
               do_augment=False
               ):
    """
    Instantiate the Framed class, create train and val dfs.

    The extract_framed() method is automatically called when the class is
    instantiated to extract the framed audios and add to the train and val dfs.

    Parameters:
      dataframe (pd.DataFrame): DataFrame containing audio file information.
      sr (int): Sample rate of the audio files.
      window_size_s (float): Window size in seconds for framing the audio.
      hop_size_s (float): Hop size in seconds for framing the audio.
      augment (bool): Whether to apply augmentation to the audio.
    """
    # instantiate class
    self.sr = sr
    self.frame_length = int(window_size_s * sr)
    self.frame_step = int(hop_size_s * sr)

    # split the dataframe to train and val by calling the _split_train_val() method
    self.train_df, self.val_df = self._split_train_val(dataframe)

    # add framed audios to train and val dfs by calling the extract_framed() method
    # only training data could be augmented, validation is never augmented
    self.extract_framed(self.train_df, augment=do_augment)
    self.extract_framed(self.val_df, augment=False)


  ########################################
  # _split_train_val function is called when the class is instantiated
  ########################################
  def _split_train_val(self,
                       dataframe):
    """
    Split the dataframe to train and val based on predefined 'set' column.

    Parameters:
      dataframe (pd.DataFrame): DataFrame containing audio file information.

    Returns:
      tuple: Tuple of train and val dataframes containing audio file information.
    """
    # extract the train set from dataframe & shuffle the train set
    train_df = dataframe[dataframe['set'] == 'train']
    train_df = train_df.sample(frac=1, random_state=1234)

    # extract the val set from dataframe & shuffle the val set
    val_df = dataframe[dataframe['set'] == 'val']
    val_df = val_df.sample(frac=1, random_state=1234)

    return train_df, val_df


  ########################################
  # augment function is called by extract_framed function if do_augment=True
  ########################################
  def augment(self, audio_array):
    """
    Apply random augmentation if needed.

    Parameters:
      audio_array (npy): Training audio array.

    Returns:
      audio_array (npy): Augmented audio array.
    """
    choice = np.random.choice(['original','noise','shift','pitch'])

    # if the audio is short (less than 7 seconds in duration), stretch it first
    if len(audio_array) <= 7*self.sr:
      audio_array = librosa.effects.time_stretch(y=audio_array, rate=np.random.uniform(low=0, high=1))
    # if the audio is not short, apply a random augmentation at random magnitude
    elif choice == 'noise':
      noise_amp = np.random.normal(loc=0.0, scale=0.05)*np.amax(audio_array)
      audio_array = audio_array + noise_amp*np.random.normal(size=audio_array.shape[0])
    elif choice == 'shift':
      shift_range = int(np.random.uniform(low=-5, high=5)*1000)
      audio_array = np.roll(audio_array, shift_range)
    elif choice == 'pitch':
      audio_array = librosa.effects.pitch_shift(y=audio_array, sr=self.sr, n_steps=np.random.uniform(low=0, high=1))
    else:
      audio_array = audio_array

    return audio_array


  ########################################
  # extract_framed function is called when the class is instantiated to extract the framed audios
  ########################################
  def extract_framed(self, dataframe, augment):
    """
    Load audio files from the given DataFrame, extract framed audios,
    and add the framed audios to the given DataFrame

    Parameters:
      dataframe (pd.DataFrame): DataFrame containing audio file information.
      augment (bool): Whether to apply augmentation to the audio.
    """
    # load audios
    audios = []
    for filename in dataframe['filename_npy']:
      audio = np.load('/content/drive/MyDrive/project/train_npy/' + filename)
      # if augment is true, call the augment function to apply augmentation to the audio
      if augment:
        audio = self.augment(audio)
      audios.append(audio)
    assert len(audios) == len(dataframe)

    # extract framed audios
    framed = []
    for audio in audios:
      framed_audio = tf.signal.frame(audio, self.frame_length, self.frame_step, pad_end=False)
      framed.append(framed_audio)
    assert len(framed) == len(audios)

    # add framed audios to df
    dataframe['framed'] = framed

# Load csv data

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/project/train_val.csv')

df.head()

,primary_label,type,latitude,longitude,rating,filename,duration,country,continent,filename_npy,set
0,comsan,call,51.9612,5.8104,2.0,comsan/XC528482.ogg,18.233500,NL,EUROPE,comsan/XC528482.npy,train
1,comsan,call,51.4522,-9.8189,4.0,comsan/XC667665.ogg,6.556000,IE,EUROPE,comsan/XC667665.npy,train
2,comsan,call,49.4452,22.6154,4.0,comsan/XC578772.ogg,31.268563,PL,EUROPE,comsan/XC578772.npy,train
3,comsan,blank,53.9299,-2.9833,2.5,comsan/XC702785.ogg,55.431875,GB,EUROPE,comsan/XC702785.npy,train
4,comsan,call,37.1379,-7.6303,3.0,comsan/XC494789.ogg,5.760000,PT,EUROPE,comsan/XC494789.npy,train


In [ ]:
len(df)

1044

# Extract train and val dfs with framed audios

Drop the audios less than 8 seconds in length first

In [ ]:
filtered = df[df['duration'] < 8.0]

filtered

,primary_label,type,latitude,longitude,rating,filename,duration,country,continent,filename_npy,set
1,comsan,call,51.4522,-9.8189,4.0,comsan/XC667665.ogg,6.556000,IE,EUROPE,comsan/XC667665.npy,train
4,comsan,call,37.1379,-7.6303,3.0,comsan/XC494789.ogg,5.760000,PT,EUROPE,comsan/XC494789.npy,train
5,comsan,blank,37.0218,-8.0121,5.0,comsan/XC750322.ogg,5.485000,PT,EUROPE,comsan/XC750322.npy,train
8,comsan,call,49.2816,-0.2060,1.0,comsan/XC548997.ogg,5.955938,FR,EUROPE,comsan/XC548997.npy,train
9,comsan,call,50.1191,18.9721,3.0,comsan/XC554039.ogg,6.144000,PL,EUROPE,comsan/XC554039.npy,train
...,...,...,...,...,...,...,...,...,...,...,...
1023,comsan,call,41.2243,1.7254,5.0,comsan/XC642258.ogg,7.288000,ES,EUROPE,comsan/XC642258.npy,val
1024,barswa,blank,51.6467,16.0796,3.5,barswa/XC743726.ogg,7.080000,PL,EUROPE,barswa/XC743726.npy,val
1031,comsan,call,51.4812,-9.7785,2.0,comsan/XC686344.ogg,6.739000,IE,EUROPE,comsan/XC686344.npy,val
1042,comsan,call,59.3794,24.2407,3.0,comsan/XC579372.ogg,7.105000,EE,EUROPE,comsan/XC579372.npy,val


In [ ]:
df.drop(filtered.index, inplace=True)

In [ ]:
assert len(df[df['duration'] < 8.0]) == 0

In [ ]:
%%time

framed = Framed(df,
               window_size_s=8.0,
               hop_size_s=4.0,
               do_augment=False)

CPU times: user 11.4 s, sys: 8.54 s, total: 19.9 s
Wall time: 3min 22s


# Save the train_df with framed audios for future use

In [ ]:
framed.train_df.head()

,primary_label,type,latitude,longitude,rating,filename,duration,country,continent,filename_npy,set,framed
668,barswa,blank,-33.2377,26.9574,5.0,barswa/XC713037.ogg,58.331000,ZA,AFRICA,barswa/XC713037.npy,train,"((tf.Tensor(-1.3069126e-05, shape=(), dtype=fl..."
247,eaywag1,call,54.1191,13.3760,3.5,eaywag1/XC658750.ogg,49.680000,DE,EUROPE,eaywag1/XC658750.npy,train,"((tf.Tensor(-1.1132361e-06, shape=(), dtype=fl..."
620,barswa,song,35.0573,34.0009,5.0,barswa/XC405617.ogg,114.207375,CY,ASIA,barswa/XC405617.npy,train,"((tf.Tensor(3.5154517e-06, shape=(), dtype=flo..."
86,comsan,call,43.5352,-1.4475,4.5,comsan/XC580687.ogg,59.820000,FR,EUROPE,comsan/XC580687.npy,train,"((tf.Tensor(-7.335485e-06, shape=(), dtype=flo..."
450,eaywag1,song,43.5118,3.8367,3.0,eaywag1/XC567939.ogg,10.031000,FR,EUROPE,eaywag1/XC567939.npy,train,"((tf.Tensor(7.8262474e-07, shape=(), dtype=flo..."


In [ ]:
with open('/content/drive/MyDrive/project/train_val_csv_pkl/train_df_8_sec.pkl', 'wb') as file:
  pickle.dump(framed.train_df, file)

del file

## reload the pickle files to confirm they are the same as the original df

In [ ]:
with open('/content/drive/MyDrive/project/train_val_csv_pkl/train_df_8_sec.pkl', 'rb') as file:
  loaded_train_df = pickle.load(file)

del file

In [ ]:
loaded_train_df.head()

,primary_label,type,latitude,longitude,rating,filename,duration,country,continent,filename_npy,set,framed
668,barswa,blank,-33.2377,26.9574,5.0,barswa/XC713037.ogg,58.331000,ZA,AFRICA,barswa/XC713037.npy,train,"((tf.Tensor(-1.3069126e-05, shape=(), dtype=fl..."
247,eaywag1,call,54.1191,13.3760,3.5,eaywag1/XC658750.ogg,49.680000,DE,EUROPE,eaywag1/XC658750.npy,train,"((tf.Tensor(-1.1132361e-06, shape=(), dtype=fl..."
620,barswa,song,35.0573,34.0009,5.0,barswa/XC405617.ogg,114.207375,CY,ASIA,barswa/XC405617.npy,train,"((tf.Tensor(3.5154517e-06, shape=(), dtype=flo..."
86,comsan,call,43.5352,-1.4475,4.5,comsan/XC580687.ogg,59.820000,FR,EUROPE,comsan/XC580687.npy,train,"((tf.Tensor(-7.335485e-06, shape=(), dtype=flo..."
450,eaywag1,song,43.5118,3.8367,3.0,eaywag1/XC567939.ogg,10.031000,FR,EUROPE,eaywag1/XC567939.npy,train,"((tf.Tensor(7.8262474e-07, shape=(), dtype=flo..."


In [ ]:
columns = ['primary_label', 'type', 'latitude', 'longitude', 'rating', 'filename', 'duration', 'country', 'filename_npy', 'set']

In [ ]:
assert framed.train_df[columns].equals(loaded_train_df[columns])

Tensor objects cannot be directly compared using pd.equals() so they are compared one by one in below script

In [ ]:
count = 0

for index in range(0,len(framed.train_df)):
  if not np.any(tf.equal(framed.train_df['framed'].iloc[index], loaded_train_df['framed'].iloc[index])):
    count +=1

assert count == 0

# Save the val_df with framed audios for future use

In [ ]:
framed.val_df.head()

,primary_label,type,latitude,longitude,rating,filename,duration,country,continent,filename_npy,set,framed
991,barswa,song,39.4709,-8.0205,3.5,barswa/XC624409.ogg,10.008000,PT,EUROPE,barswa/XC624409.npy,val,"((tf.Tensor(3.148432e-08, shape=(), dtype=floa..."
760,barswa,call,51.4522,-9.8189,5.0,barswa/XC666943.ogg,11.885000,IE,EUROPE,barswa/XC666943.npy,val,"((tf.Tensor(-8.617062e-07, shape=(), dtype=flo..."
876,comsan,call,50.7347,3.2143,4.0,comsan/XC578171.ogg,21.707000,BE,EUROPE,comsan/XC578171.npy,val,"((tf.Tensor(-3.9801816e-06, shape=(), dtype=fl..."
927,eaywag1,song,46.0192,61.8460,5.0,eaywag1/XC184434.ogg,63.921625,KZ,ASIA,eaywag1/XC184434.npy,val,"((tf.Tensor(-4.7415142e-07, shape=(), dtype=fl..."
880,comsan,call,42.9037,13.9077,4.0,comsan/XC433334.ogg,24.006000,IT,EUROPE,comsan/XC433334.npy,val,"((tf.Tensor(-1.1874385e-05, shape=(), dtype=fl..."


In [ ]:
with open('/content/drive/MyDrive/project/train_val_csv_pkl/val_df_8_sec.pkl', 'wb') as file:
  pickle.dump(framed.val_df, file)

## reload the pickle files to confirm they are the same as the original df

In [ ]:
with open('/content/drive/MyDrive/project/train_val_csv_pkl/val_df_8_sec.pkl', 'rb') as file:
  loaded_val_df = pickle.load(file)

In [ ]:
loaded_val_df.head()

,primary_label,type,latitude,longitude,rating,filename,duration,country,continent,filename_npy,set,framed
991,barswa,song,39.4709,-8.0205,3.5,barswa/XC624409.ogg,10.008000,PT,EUROPE,barswa/XC624409.npy,val,"((tf.Tensor(3.148432e-08, shape=(), dtype=floa..."
760,barswa,call,51.4522,-9.8189,5.0,barswa/XC666943.ogg,11.885000,IE,EUROPE,barswa/XC666943.npy,val,"((tf.Tensor(-8.617062e-07, shape=(), dtype=flo..."
876,comsan,call,50.7347,3.2143,4.0,comsan/XC578171.ogg,21.707000,BE,EUROPE,comsan/XC578171.npy,val,"((tf.Tensor(-3.9801816e-06, shape=(), dtype=fl..."
927,eaywag1,song,46.0192,61.8460,5.0,eaywag1/XC184434.ogg,63.921625,KZ,ASIA,eaywag1/XC184434.npy,val,"((tf.Tensor(-4.7415142e-07, shape=(), dtype=fl..."
880,comsan,call,42.9037,13.9077,4.0,comsan/XC433334.ogg,24.006000,IT,EUROPE,comsan/XC433334.npy,val,"((tf.Tensor(-1.1874385e-05, shape=(), dtype=fl..."


In [ ]:
assert framed.val_df[columns].equals(loaded_val_df[columns])

In [ ]:
count = 0

for index in range(0,len(framed.val_df)):
  if not np.any(tf.equal(framed.val_df['framed'].iloc[index], loaded_val_df['framed'].iloc[index])):
    count +=1

assert count == 0